In [83]:
using LinearAlgebra
using ForwardDiff
using SatelliteDynamics
using DifferentialEquations
using Plots
using ECOS, Convex
using Mosek, MosekTools

In [84]:
#From slide deck

#values are in km and degrees 

#orbits elements of und roads 1
#last is the true anomaly 
elements1 = [6885.635e3, 0.001089, 97.43, 0, 0, 0]


#orbital elements of und roads 2
#last is the true anomaly
elements2 = [6885.635e3, 0.001089, 97.43, 0, 0, 0.016667]

6-element Vector{Float64}:
  6.885635e6
  0.001089
 97.43
  0.0
  0.0
  0.016667

In [85]:
#convert the true anomaly to mean anomaly to be able to do the conversion and degrees to radians

In [86]:
function degrees_to_rad(x)

    return x*pi/180
    
end

degrees_to_rad (generic function with 1 method)

In [87]:
#v is true anomaly 
#e is the eccentricity
function true_anom_to_eccentric_anom(v, e)

    E = 2*atand(sqrt((1-e)/(1+e))*tand(v/2))

    #eccentric anomaly
    return E

end

true_anom_to_eccentric_anom (generic function with 1 method)

In [88]:
function eccentric_anom_to_mean_anom(E, e)

    M = E - e*sind(E)

    #mean anomaly
    return M

end

eccentric_anom_to_mean_anom (generic function with 1 method)

In [89]:
#eccentric anomaly sat 1
E1 = true_anom_to_eccentric_anom(elements1[end], elements1[2])

#mean anomaly sat 1
M1 = eccentric_anom_to_mean_anom(E1, elements1[2])

0.0

In [90]:
#eccentric anomaly sat 2
E2 = true_anom_to_eccentric_anom(elements2[end], elements2[2])

#eccentric anomaly sat 2
M2  = eccentric_anom_to_mean_anom(E2, elements2[2])

0.016648543070573774

In [91]:
#get all the units consistent (m, radians)
#need to change the inclanation to radians, the rest are zeros

elements1_m = [elements1[1:2]; degrees_to_rad(elements1[3]); elements1[4:5]; degrees_to_rad(M1)]

elements2_m = [elements2[1:2]; degrees_to_rad(elements2[3]); elements2[4:5]; degrees_to_rad(M2)]

6-element Vector{Float64}:
 6.885635e6
 0.001089
 1.7004742902180754
 0.0
 0.0
 0.00029057189224159907

In [92]:
#state satellite 1 (target)

x_1 = sOSCtoCART(elements1_m, use_degrees=false)

6-element Vector{Float64}:
    6.878136543485e6
    0.0
    0.0
    0.0
 -984.9589353739226
 7552.79897887146

In [93]:
#state satellite 2 (chaser)

x_2 = sOSCtoCART(elements2_m, use_degrees=false)

6-element Vector{Float64}:
    6.878136252166805e6
 -259.0113696278884
 1986.134383662354
   -2.2156282462577286
 -984.9588936567426
 7552.798658978454

In [94]:
#relative state of chaser wrt target
#this is in ECI frame
difference = x_2 - x_1 

6-element Vector{Float64}:
   -0.2913181949406862
 -259.0113696278884
 1986.134383662354
   -2.2156282462577286
    4.171718001089175e-5
   -0.0003198930062353611

In [95]:
#this is in the LVLH (RTN) frame of the target (primary observing satellite) 
initial_difference = sECItoRTN(x_1, x_2) 

6-element StaticArraysCore.SVector{6, Float64} with indices SOneTo(6):
   -0.2913181949406862
 2002.9519913273894
    2.7017514072632172e-14
    0.002412819262479271
   -1.5962127995240716e-12
    3.511794450825904e-14

In [96]:
position_scale = norm(x_1[1:3])

6.878136543485e6

In [97]:
#bake in the scaling into the B matrix
#have 4.66 mm/s2 be equal to 1 distance_unit/time_unit

In [98]:
#u is scaled to 1

In [99]:
#first attempt is by only scaling time
tu = sqrt(4.66e-3)

0.06826419266350404

In [100]:
#scale the distance by the l2 norm of the target

#scale the time by the period of the target

In [101]:
# velocity_scale = 1/tu
# acceleration_scale = 1/(tu)^2

In [102]:
#this is what goes in the B matrix (not working)
#1/acceleration_scale

In [103]:
#to get it to equal 1 you multiply

In [104]:
#mu - gravitational parameter (m3/s2)
mu = 3.986e14

3.986e14

In [105]:
#radius of Earth (m)
r_e = 6.371e6

6.371e6

In [106]:
#sma from ppt (m)
a = 6.85635e6

6.85635e6

In [107]:
#constant mean motion of target n = sqrt(mu/a^3)
#units: rad/s
n = sqrt(mu/a^3)

0.0011120624547982091

In [108]:
#number of states
nx = 6

#number of control inputs
nu = 3

3

In [109]:
#this is a whole orbit for dt=60
N = 94

#this is a whole orbit for dt=1
#N = 5650

94

In [110]:
#target orbit period in seconds

#this a is of the target orbit 

T = 2*pi*sqrt((a^3)/(mu))

5650.029168838104

In [111]:
#time scale
time_scale = T

5650.029168838104

In [112]:
velocity_scale = position_scale/time_scale 

acceleration_scale = position_scale/(time_scale^2)

0.2154613681741382

In [113]:
n_scaled = n/time_scale 

1.9682419710886172e-7

In [114]:
#update code to use the scaled units!

In [115]:
#timestep 
#works better with 1 second...
#dt = 1

#trying out
dt = 60

60

In [116]:
dt_scaled = 1/time_scale 

0.00017699023670804238

In [117]:
thist = LinRange(1, N*dt, N)

94-element LinRange{Float64, Int64}:
 1.0, 61.6344, 122.269, 182.903, 243.538, …, 5458.1, 5518.73, 5579.37, 5640.0

In [118]:
x0_target = zeros(nx) 

6-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [119]:
#using the true data from the ppt
#relative state of the chaser wrt target 
#in the target RTN frame
x0_chaser = initial_difference

6-element StaticArraysCore.SVector{6, Float64} with indices SOneTo(6):
   -0.2913181949406862
 2002.9519913273894
    2.7017514072632172e-14
    0.002412819262479271
   -1.5962127995240716e-12
    3.511794450825904e-14

In [120]:
x0_chaser_scaled = [x0_chaser[1:3]/position_scale; x0_chaser[4:6]/velocity_scale] 

6-element Vector{Float64}:
 -4.235423258885781e-8
  0.0002912056163270841
  3.928028166033324e-21
  1.9820047371777003e-6
 -1.3112052690384923e-15
  2.884755334048192e-17

In [121]:
A = zeros(nx, nx)
A[1:3, 4:6] = I(3)
A[4, 1] = 3*n^2
A[6, 3] = -n^2
A[4, 5] = 2*n
A[5, 4] = -2*n

-0.0022241249095964182

In [122]:
A_scaled = zeros(nx, nx)
A_scaled[1:3, 4:6] = I(3)
A_scaled[4, 1] = 3*n_scaled^2
A_scaled[6, 3] = -n_scaled^2
A_scaled[4, 5] = 2*n_scaled
A_scaled[5, 4] = -2*n_scaled

-3.9364839421772345e-7

In [123]:
#mass of the satellite (kg) from data
m = 5.22

5.22

In [124]:
#si units
B = [zeros(3,3); I(3)]

#scaled
#B = [zeros(3,3); 1/acceleration_scale*I(3)]

#trying new
#this maps an l1 norm of 1 to 4.66e-3 (bug, not working)

#B = [zeros(3,3); 1/acceleration_scale*I(3)]

6×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

In [125]:
#continous dynamics
function spacecraft_dynamics(x,u)

    xdot = A*x + B*u

    return xdot

end

spacecraft_dynamics (generic function with 1 method)

In [126]:
#discretize the dynamics model 
H_scaled = exp(dt_scaled*[A_scaled B; zeros(nu, nx+nu)])

9×9 Matrix{Float64}:
  1.0          0.0   0.0           0.00017699   …  3.63752e-19  0.0
 -4.2275e-32   1.0   0.0          -6.16563e-15     1.56628e-8   0.0
  0.0          0.0   1.0           0.0             0.0          1.56628e-8
  2.05697e-17  0.0   0.0           1.0             6.16563e-15  0.0
 -7.16565e-28  0.0   0.0          -6.96719e-11     0.00017699   0.0
  0.0          0.0  -6.85656e-18   0.0          …  0.0          0.00017699
  0.0          0.0   0.0           0.0             0.0          0.0
  0.0          0.0   0.0           0.0             1.0          0.0
  0.0          0.0   0.0           0.0             0.0          1.0

In [127]:
#Scaled Discrete Dynamics Matrices
Ad_scaled  = H_scaled[1:nx, 1:nx]
Bd_scaled = H_scaled[1:nx, (nx+1):end]

6×3 Matrix{Float64}:
  1.56628e-8   3.63752e-19  0.0
 -3.63752e-19  1.56628e-8   0.0
  0.0          0.0          1.56628e-8
  0.00017699   6.16563e-15  0.0
 -6.16563e-15  0.00017699   0.0
  0.0          0.0          0.00017699

In [128]:
#discretize the dynamics model 
H = exp(dt*[A B; zeros(nu, nx+nu)])

9×9 Matrix{Float64}:
  1.00668      0.0   0.0         …  1799.33       80.0507      0.0
 -0.000296992  1.0   0.0             -80.0507   1797.33        0.0
  0.0          0.0   0.997775          0.0         0.0      1799.33
  0.000222438  0.0   0.0              59.9555      4.00194     0.0
 -1.48474e-5   0.0   0.0              -4.00194    59.822       0.0
  0.0          0.0  -7.41459e-5  …     0.0         0.0        59.9555
  0.0          0.0   0.0               1.0         0.0         0.0
  0.0          0.0   0.0               0.0         1.0         0.0
  0.0          0.0   0.0               0.0         0.0         1.0

In [129]:
#Discrete Dynamics Matrices
Ad  = H[1:nx, 1:nx]
Bd = H[1:nx, (nx+1):end]

6×3 Matrix{Float64}:
 1799.33       80.0507      0.0
  -80.0507   1797.33        0.0
    0.0         0.0      1799.33
   59.9555      4.00194     0.0
   -4.00194    59.822       0.0
    0.0         0.0        59.9555

In [130]:
Ad_scaled 

6×6 Matrix{Float64}:
  1.0          0.0   0.0           0.00017699   6.16563e-15  0.0
 -4.2275e-32   1.0   0.0          -6.16563e-15  0.00017699   0.0
  0.0          0.0   1.0           0.0          0.0          0.00017699
  2.05697e-17  0.0   0.0           1.0          6.96719e-11  0.0
 -7.16565e-28  0.0   0.0          -6.96719e-11  1.0          0.0
  0.0          0.0  -6.85656e-18   0.0          0.0          1.0

In [131]:
#discrete dynamics
function spacecraft_dynamics_discrete(x,u)

    xnext = Ad*x + Bd*u

    return xnext

end

spacecraft_dynamics_discrete (generic function with 1 method)

In [132]:
acceleration_scale 

0.2154613681741382

In [133]:
4.66e-3/acceleration_scale 

0.0216280070970019

In [134]:
#both of these horizons are 1 orbit...

#when dt is a minute
#N_h = 94

#when dt is a second
#N_h = 5650


N_h = N

94

In [135]:
x_initial = x0_chaser

6-element StaticArraysCore.SVector{6, Float64} with indices SOneTo(6):
   -0.2913181949406862
 2002.9519913273894
    2.7017514072632172e-14
    0.002412819262479271
   -1.5962127995240716e-12
    3.511794450825904e-14

In [136]:
x_goal = x0_target

6-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [137]:
test_og = Ad*x0_chaser

6-element Vector{Float64}:
   -0.14860116219705305
 2002.9424218890772
    2.132470941681533e-12
    0.002342650074188926
   -0.0003174205091450142
    3.5037796697226343e-14

In [ ]:
test = Ad_scaled*x0_chaser_scaled  

In [ ]:
function update_prob(x_initial_k)


    X = Convex.Variable(nx, N_h)
    U = Convex.Variable(nu, N_h-1)
    
    #Initial State Constraint
    cons = Constraint[X[:,1] == x_initial_k]

    #Dynamics Constraint
    for k=1:(N_h-1)

        push!(cons, X[:,k+1] - (Ad*X[:,k] + Bd*U[:,k]) == zeros(6))

    end

    #add in thrust constraints

    #try both l1 and l2
    for k=1:(N_h-1)

    #    #this thrust is small
    #    #trying to increase it by an order of magnitude 

       #L1 works
       push!(cons, norm(U[:,k],2) <= 4.6e-3)

    #    #when scaled (not working)
    #    #push!(cons, norm(U[:,k],1) <= 1.0)


       #bound constraints (working)
    #    push!(cons, U[1,k] <= 4.6e-3)
    #    push!(cons, U[2,k] <= 4.6e-3)
    #    push!(cons, U[3,k] <= 4.6e-3)
    #    push!(cons, U[1,k] >= -4.6e-3)
    #    push!(cons, U[2,k] >= -4.6e-3)
    #    push!(cons, U[3,k] >= -4.6e-3)

    end




    return cons, X, U

end

In [ ]:
function solve_prob(cons, X, U, N)


    #objective 
    obj = 0

    #weights depending how conservative you want to be on fuel vs reaching the goal
    beta = 100
    alpha = 1

    for k=1:N-1
        obj += beta*norm(U[:,k], 1) + alpha*norm(X[:,k], 2)
    end

    #terminal cost
    obj += alpha*norm(X[:,N], 2)

    prob = minimize(obj, cons)

    #solve with ECOS
    #Convex.solve!(prob, ()-> ECOS.Optimizer(), silent_solver=true);

    Convex.solve!(prob, ()-> Mosek.Optimizer(), silent_solver = true)

    Xm = X.value
    Um = U.value

    return Xm, Um

end

In [ ]:
#iterations = 1000
num_orbits = 2

#50 orbits worth of data
iterations = N*num_orbits

In [ ]:
#set up in Convex.jl

In [ ]:
xhist = zeros(nx, iterations)
uhist = zeros(nu, iterations-1)

xhist[:,1] = x_initial

In [ ]:
# for i=1:iterations-1

    i = 1
    cons, X, U = update_prob(xhist[:,i])

    Xm, Um = solve_prob(cons, X, U, N_h)

    #take one step 
    uhist[:,i] = Um[:,1]

    xhist[:,i+1] = spacecraft_dynamics_discrete(xhist[:,i], uhist[:,i])


# end

In [ ]:
all_norms = zeros(iterations-1)

for i=1:iterations-1

    all_norms[i] = norm(uhist[:,i],1)

end

In [ ]:
uhist 

In [ ]:
#does full on norm for all the iterations...
plot(all_norms) 
#plot!(4.6e-3*ones(iterations))

In [ ]:
all_norms 


In [ ]:
xhist 

In [ ]:
plot(xhist[1,:], xhist[2,:], xhist[3,:], label = "Chaser Trajectory")

scatter!([xhist[1,1]], [xhist[2,1]], [xhist[3,1]], label = "Initial State")
scatter!([xhist[1,end]], [xhist[2,end]], [xhist[3,end]], label = "End State")

scatter!([0], [0], [0], label= "Target")

In [ ]:
plot(xhist[2,:], xhist[1,:], xlabel= "Tangential (m)", ylabel="Radial (m)")

In [ ]:
plot(xhist[3,:], xhist[1,:], xlabel= "Normal (m)", ylabel="Radial (m)", label=false)

In [ ]:
plot(xhist[1,:], title="Positions") 
plot!(xhist[2,:])
plot!(xhist[3,:])

In [ ]:
plot(xhist[4,:], title="Velocities") 
plot!(xhist[5,:])
plot!(xhist[6,:])

In [ ]:
plot(uhist[1,:], title = "Controls")
plot!(uhist[2,:])
plot!(uhist[3,:])

In [ ]:
plot(uhist[1,1:50], title = "Controls")
plot!(uhist[2,1:50])
plot!(uhist[3,1:50])

In [ ]:
fuel_usage = sum(abs.(vec(uhist)))

In [ ]:
dt*fuel_usage 

In [ ]:
no_control = zeros(nx, iterations)

In [ ]:
no_control[:,1] = x0_chaser

In [ ]:
for i = 1:iterations-1

    no_control[:, i+1] = Ad*no_control[:,i]

end

In [ ]:
# plot(no_control[1,:], no_control[2,:], no_control[3,:], zlims=(-1,1))
# scatter!([0], [0], [0])

In [ ]:
plot(no_control[2,:], no_control[1,:])

In [ ]:
plot(no_control[3,:], no_control[1,:])